In [1]:
import os
import sys
import pandas as pd
src_dir = os.path.abspath('../src')
if src_dir not in sys.path:
    sys.path.append(src_dir)

In [2]:
import pandas as pd
import asyncio

import gspread_asyncio
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

from datetime import datetime
import pytz
import os

from utils import md_table_to_df


# Файл для сервиcного аккаунта
SRC_DIR = "/Users/V/Desktop/pets/Agro/src"
SERVICE_ACCOUNT_FILE = f'{SRC_DIR}/progressagroproject-9dfb76fd1abc.json'
FOLDER_ID = '1EEbdC6WDtSpy68pmNLHAeU4d7Ca01MdK'

SCOPES = ['https://www.googleapis.com/auth/spreadsheets',
          'https://www.googleapis.com/auth/drive']

# Аутентификация
def get_creds():
    return Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=SCOPES
    )

drive_service = build('drive', 'v3', credentials=get_creds())
# Выносим создание клиента в корень, чтобы ускорить функцию
agcm = gspread_asyncio.AsyncioGspreadClientManager(get_creds)
client = await agcm.authorize()

In [3]:
msk_tz = pytz.timezone('Europe/Moscow')
today_str = datetime.now(msk_tz).strftime('%d.%m.%y')
report_name = f"Отчет_{today_str}"
report_name

'Отчет_17.04.25'

In [4]:
spreadsheet = await client.open(report_name)

In [7]:
file_metadata = await asyncio.to_thread(
    lambda: drive_service.files().get(
        fileId=spreadsheet.id,
#         fileId = '1rqF6Sy2R7WX1RC71Ota_ZUA_h0xKmTMekJDNOx0uhRc',
        fields="parents,trashed"
    ).execute()
)

In [8]:
file_metadata

{'trashed': False, 'parents': ['0AIDRnhNHOzW5Uk9PVA']}

In [21]:
if FOLDER_ID not in file_metadata['parents']:
    await asyncio.to_thread(
        lambda: drive_service.files().delete(fileId=spreadsheet.id).execute()
    )